# Download and store data

This notebook contains information on downloading the Quandl Wiki stock prices and a few other sources that we use throughout the book. 

## Imports & Settings

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
from pathlib import Path
import requests
from io import BytesIO
from zipfile import ZipFile, BadZipFile

import numpy as np
import pandas as pd
import pandas_datareader.data as web
from sklearn.datasets import fetch_openml
import datetime
import yfinance as yf

pd.set_option('display.expand_frame_repr', False)

## Set Data Store path

Modify path if you would like to store the data elsewhere and change the notebooks accordingly

In [4]:
DATA_STORE = Path('assets.h5')

## YAHOO Prices

Using yahoo finance API for python to download the stock prices.

Get the list of tickers in MERVAL.

In [6]:
import requests
from bs4 import BeautifulSoup

# get all components of merval index
url = "https://es.wikipedia.org/wiki/S%26P_Merval"

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

panel_lider_section = soup.find(id="Empresas_que_participan_Panel_Líder")
lider_table = panel_lider_section.find_next("table")

ba_tickers = []
for row in lider_table.find_all('tr')[1:]:  # Skip the header row
    cols = row.find_all('td')
    ticker = cols[0].text.strip()
    ba_tickers.append(ticker + ".BA")
    
panel_lider_section = soup.find(id="Empresas_panel_general")
general_table = panel_lider_section.find_next("table")

for row in general_table.find_all('tr')[1:]:  # Skip the header row
    cols = row.find_all('td')
    ticker = cols[0].text.strip()
    ba_tickers.append(ticker + ".BA")

len(ba_tickers)

62

In [15]:
list(set(ba_tickers).difference(set(['ESME.BA', 'PGR.BA', 'RICH.BA', 'TGLT.BA'])))

['MIRG.BA',
 'MOLA.BA',
 'TXAR.BA',
 'CGPA2.BA',
 'INTR.BA',
 'YPFD.BA',
 'SEMI.BA',
 'LONG.BA',
 'CADO.BA',
 'VALO.BA',
 'TRAN.BA',
 'HAVA.BA',
 'CTIO.BA',
 'METR.BA',
 'CEPU.BA',
 'BHIP.BA',
 'AUSO.BA',
 'LEDE.BA',
 'OEST.BA',
 'TECO2.BA',
 'MOLI.BA',
 'CELU.BA',
 'INVJ.BA',
 'POLL.BA',
 'ROSE.BA',
 'TGSU2.BA',
 'IRSA.BA',
 'DGCU2.BA',
 'SUPV.BA',
 'CARC.BA',
 'CAPX.BA',
 'PAMP.BA',
 'FERR.BA',
 'TGNO4.BA',
 'PATA.BA',
 'EDN.BA',
 'GCLA.BA',
 'MTR.BA',
 'SAMI.BA',
 'BMA.BA',
 'BPAT.BA',
 'GARO.BA',
 'MORI.BA',
 'BYMA.BA',
 'CECO2.BA',
 'GGAL.BA',
 'GRIM.BA',
 'GBAN.BA',
 'LOMA.BA',
 'DOME.BA',
 'DYCA.BA',
 'BBAR.BA',
 'ALUA.BA',
 'COME.BA',
 'FIPL.BA',
 'GAMI.BA',
 'BOLT.BA',
 'CRES.BA']

Download the historical data FROM 2010 TO 2023.

In [21]:
import yfinance as yf
import pandas as pd

# store prices data
all_data = pd.DataFrame()
for symbol in ba_tickers:
    try:
        stock_data = yf.download(symbol, start="2000-01-01", end="2024-07-28", progress=False)
        stock_data['ticker'] = symbol
        stock_data['open'] = stock_data['Open']
        stock_data['high'] = stock_data['High']
        stock_data['low'] = stock_data['Low']
        stock_data['close'] = stock_data['Close']
        stock_data['volume'] = stock_data['Volume']
        stock_data['adj_open'] = stock_data['Open']
        stock_data['adj_high'] = stock_data['High']
        stock_data['adj_low'] = stock_data['Low']
        stock_data['adj_close'] = stock_data['Adj Close']
        stock_data['adj_volume'] = stock_data['Volume']
        stock_data['ex-dividend'] = 0  # Placeholder as Yahoo Finance does not provide this
        stock_data['split_ratio'] = 1  # Placeholder as Yahoo Finance does not provide this
        stock_data.reset_index(inplace=True)
        stock_data['date'] = stock_data['Date']
        
        stock_data = stock_data[['ticker', 'date', 'open', 'high', 'low', 'close', 'volume',
                                 'ex-dividend', 'split_ratio', 'adj_open', 'adj_high', 'adj_low',
                                 'adj_close', 'adj_volume']]
        
        all_data = pd.concat([all_data, stock_data])
    except Exception as e:
        ba_tickers.remove(symbol)
        print(f"Failed to download data for {symbol}: {e}")

csv_output_file = 'merval_stock_prices.csv'
all_data.to_csv(csv_output_file, index=False)
print(f"All data saved to {csv_output_file}")


1 Failed download:
['ESME.BA']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')

1 Failed download:
['PGR.BA']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')

1 Failed download:
['RICH.BA']: YFInvalidPeriodError("%ticker%: Period 'max' is invalid, must be one of ['1d', '5d']")

1 Failed download:
['TGLT.BA']: YFTzMissingError('$%ticker%: possibly delisted; No timezone found')


All data saved to merval_stock_prices.csv


Store data in h5 format

In [22]:
df = (pd.read_csv('merval_stock_prices.csv',
                 parse_dates=['date'],
                 index_col=['date', 'ticker'],
                 infer_datetime_format=True)
     .sort_index())

with pd.HDFStore(DATA_STORE) as store:
    store.put('merval/prices', df)

### Prices Metadata

Download metadata from yahoo.

In [ ]:
from datetime import datetime
# store metadata
stock_metadata = []
stock_names = []
for symbol in ba_tickers:
    try:
        stock = yf.Ticker(symbol)
        info = stock.info
        data = {
            "ticker": symbol,
            "name": info.get("longName", ""),
            "lastsale": info.get("regularMarketPreviousClose", None),
            "marketcap": info.get("marketCap", None),
            "ipoyear": datetime.fromtimestamp(info.get("governanceEpochDate", -1)).year if info.get("governanceEpochDate", 0) >= 0 else None,
            "sector": info.get("sector", ""),
            "industry": info.get("industry", "")
        }
        names = {
            "ticker": symbol,
            "name": info.get("longName", "")
        }
        stock_metadata.append(data)
        stock_names.append(names)
    except Exception as e:
        print(f"Failed to download metadata for {symbol}: {e}")

names_df = pd.DataFrame(stock_names)
names_df.to_csv('merval_stock_names.csv', index=False)

names_df = pd.DataFrame(stock_metadata)
names_df.to_csv('merval_stock_metadata.csv', index=False)

Store data in h5 format

In [ ]:
df = pd.read_csv('merval_stock_metadata.csv')

with pd.HDFStore(DATA_STORE) as store:
    store.put('merval/stocks', df)

## S&P Merval Prices

The following code downloads historical S&P Merval prices from yahoo finance.

In [ ]:
# Fetch data
merval_data = yf.download('^MERV', start="2010-01-01", end="2024-01-01", progress=False)

# Display the fetched data
print(merval_data.head())

              Open    High     Low   Close  Adj Close  Volume
Date                                                         
2010-01-04  2330.0  2394.0  2330.0  2385.0     2385.0       0
2010-01-05  2385.0  2411.0  2385.0  2402.0     2402.0       0
2010-01-06  2400.0  2401.0  2347.0  2363.0     2363.0       0
2010-01-07  2363.0  2393.0  2332.0  2390.0     2390.0       0
2010-01-08  2378.0  2378.0  2335.0  2353.0     2353.0       0


In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('spmerval', merval_data)

## Metadata on traded companies

In [ ]:
df = pd.read_csv('merval_stock_metadata.csv')
df.head()

,ticker,name,lastsale,marketcap,ipoyear,sector,industry
0,ALUA.BA,Aluar Aluminio Argentino S.A.I.C.,1050.0,2.982000e+12,1969,Basic Materials,Aluminum
1,BBAR.BA,Banco BBVA Argentina S.A.,4065.0,2.475348e+12,1969,Financial Services,Banks - Regional
2,BMA.BA,Banco Macro S.A.,7420.0,4.686208e+12,1969,Financial Services,Banks - Regional
3,BYMA.BA,Bolsas y Mercados Argentinos S.A.,366.0,1.385844e+12,1969,Financial Services,Capital Markets
4,GGAL.BA,Grupo Financiero Galicia S.A.,4110.0,6.046229e+12,1969,Financial Services,Banks - Regional


See marketcap distributions

In [ ]:
df.marketcap.describe(percentiles=np.arange(.1, 1, .1).round(1)).apply(lambda x: f'{int(x):,d}')

count                    59
mean      1,398,700,466,349
std       2,697,537,653,409
min             714,253,312
10%          12,776,256,512
20%          77,864,810,905
30%         152,979,668,991
40%         281,644,453,068
50%         392,592,457,728
60%         739,587,876,454
70%       1,030,673,714,380
80%       1,539,648,153,190
90%       3,745,141,804,236
max      15,342,576,336,896
Name: marketcap, dtype: object

Store metadata

In [ ]:
with pd.HDFStore(DATA_STORE) as store:
    store.put('merval/stocks', df.set_index('ticker'))